<a href="https://colab.research.google.com/github/cckmwong/data_analyst.github.io/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from datetime import datetime

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery
import pandas as pd

#
client = bigquery.Client(project="portfolio-454106")

# SQL Query to extract the data in Google BigQuery
query = """
SELECT
  *
FROM
  `portfolio-454106.main.customers`
"""

# Store results of the table in a Pandas DataFrame
df = client.query(query).to_dataframe()

In [4]:
df

,Customer_ID,Age,Gender,Marital_Status,Education_Level,Geographic_Information,Occupation,Income_Level,Behavioral_Data,Purchase_History,Interactions_with_Customer_Service,Insurance_Products_Owned,Coverage_Amount,Premium_Amount,Policy_Type,Customer_Preferences,Preferred_Communication_Channel,Preferred_Contact_Time,Preferred_Language
0,8,52,Male,Divorced,High School Diploma,Andaman and Nicobar Islands,Salesperson,54148,policy1,2018-01-24,In-Person,policy2,272803,2692,Business,In-Person Meeting,In-Person Meeting,Afternoon,Spanish
1,19,19,Male,Separated,Doctorate,Andaman and Nicobar Islands,Doctor,135315,policy3,2018-05-17,Mobile App,policy3,73501,4372,Individual,Email,Email,Morning,Mandarin
2,27,64,Female,Single,Associate Degree,Andaman and Nicobar Islands,Artist,54260,policy5,2020-01-13,Email,policy1,296642,1246,Family,Email,Mail,Afternoon,German
3,33,26,Female,Separated,Doctorate,Andaman and Nicobar Islands,Lawyer,132053,policy2,2020-12-16,Email,policy4,268391,1394,Individual,Mail,Email,Morning,Mandarin
4,55,18,Male,Single,High School Diploma,Andaman and Nicobar Islands,Manager,39410,policy3,2020-08-05,Mobile App,policy5,811832,700,Family,In-Person Meeting,Mail,Evening,German
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53498,53424,33,Male,Divorced,Doctorate,West Bengal,Lawyer,21919,policy3,2021-06-11,Mobile App,policy4,437617,4664,Individual,Email,Text,Evening,French
53499,53442,37,Male,Divorced,Doctorate,West Bengal,Lawyer,147879,policy3,2023-12-27,Mobile App,policy5,349207,3500,Group,Email,Text,Morning,English
53500,53448,64,Male,Married,High School Diploma,West Bengal,Lawyer,30325,policy2,2022-03-22,Chat,policy3,512172,807,Business,Text,Phone,Weekends,Spanish
53501,53470,25,Female,Single,Master's Degree,West Bengal,Doctor,146841,policy5,2020-09-05,Chat,policy3,262209,4676,Individual,In-Person Meeting,Phone,Morning,German


In [5]:
# Ensure 'Purchase_History' is in pandas datetime format
df['Purchase_History'] = pd.to_datetime(df['Purchase_History'], format='%d-%m-%Y')

# Get today's date as a pandas Timestamp
today = pd.Timestamp.today()

# Convert 'Purchase_History' to individual timestamps before subtracting
df['Years_Since_Purchase'] = (today - df['Purchase_History'].astype('datetime64[ns]')).dt.days / 365

# Round to 2 decimal places
df['Years_Since_Purchase'] = df['Years_Since_Purchase'].round(2)


In [6]:
df.columns

Index(['Customer_ID', 'Age', 'Gender', 'Marital_Status', 'Education_Level',
       'Geographic_Information', 'Occupation', 'Income_Level',
       'Behavioral_Data', 'Purchase_History',
       'Interactions_with_Customer_Service', 'Insurance_Products_Owned',
       'Coverage_Amount', 'Premium_Amount', 'Policy_Type',
       'Customer_Preferences', 'Preferred_Communication_Channel',
       'Preferred_Contact_Time', 'Preferred_Language', 'Years_Since_Purchase'],
      dtype='object')

In [7]:
df_customers = df[['Age', 'Gender', 'Marital_Status', 'Education_Level','Income_Level','Coverage_Amount', 'Premium_Amount', 'Policy_Type','Years_Since_Purchase']]

In [8]:
df_customers

,Age,Gender,Marital_Status,Education_Level,Income_Level,Coverage_Amount,Premium_Amount,Policy_Type,Years_Since_Purchase
0,52,Male,Divorced,High School Diploma,54148,272803,2692,Business,7.15
1,19,Male,Separated,Doctorate,135315,73501,4372,Individual,6.84
2,64,Female,Single,Associate Degree,54260,296642,1246,Family,5.18
3,26,Female,Separated,Doctorate,132053,268391,1394,Individual,4.25
4,18,Male,Single,High School Diploma,39410,811832,700,Family,4.62
...,...,...,...,...,...,...,...,...,...
53498,33,Male,Divorced,Doctorate,21919,437617,4664,Individual,3.77
53499,37,Male,Divorced,Doctorate,147879,349207,3500,Group,1.22
53500,64,Male,Married,High School Diploma,30325,512172,807,Business,2.99
53501,25,Female,Single,Master's Degree,146841,262209,4676,Individual,4.53


In [9]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53503 entries, 0 to 53502
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   53503 non-null  Int64  
 1   Gender                53503 non-null  object 
 2   Marital_Status        53503 non-null  object 
 3   Education_Level       53503 non-null  object 
 4   Income_Level          53503 non-null  Int64  
 5   Coverage_Amount       53503 non-null  Int64  
 6   Premium_Amount        53503 non-null  Int64  
 7   Policy_Type           53503 non-null  object 
 8   Years_Since_Purchase  53503 non-null  float64
dtypes: Int64(4), float64(1), object(4)
memory usage: 3.9+ MB


In [10]:
df_customers.describe()

,Age,Income_Level,Coverage_Amount,Premium_Amount,Years_Since_Purchase
count,53503.0,53503.0,53503.0,53503.0,53503.000000
mean,44.140945,82768.324318,492580.789638,3023.702447,4.209217
std,15.079486,36651.07567,268405.505571,1285.834295,1.729986
min,18.0,20001.0,50001.0,500.0,1.220000
25%,32.0,51568.5,249613.5,1817.0,2.720000
50%,43.0,80719.0,477261.0,3194.0,4.210000
75%,57.0,115973.5,739124.0,4311.5,5.690000
max,70.0,149999.0,1000000.0,5000.0,7.210000
